<a href="https://colab.research.google.com/github/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/11-deep-learning-for-text/03_transformer_encoder_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Transformer encoder for text classification

Factoring outputs into multiple independent spaces, adding residual connections, adding normalization layers—all of these are standard architecture patterns that one would be wise to leverage in any complex model. Together, these bells and whistles form the Transformer encoder—one
of two critical parts that make up the Transformer architecture.

<img src='https://github.com/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/11-deep-learning-for-text/images/2.png?raw=1' width='600'/>

The original Transformer architecture consists of two parts: a Transformer encoder that processes the source sequence, and a Transformer decoder that uses the source sequence to generate a translated version.

Crucially, the encoder part can be used for text classification—it’s a very generic module that ingests a sequence and learns to turn it into a more useful representation.

Let’s implement a Transformer encoder and try it on the movie review sentiment
classification task.





##Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import os, pathlib, shutil, random

Let’s start by downloading the dataset from the Stanford page and uncompressing it:

In [2]:
%%shell

curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
tar -xf aclImdb_v1.tar.gz

# delete unwanted file and subdirectory
rm -rf aclImdb/train/unsup
rm -rf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  30.2M      0  0:00:02  0:00:02 --:--:-- 30.2M


In [3]:
# take a look at the content of a few of these text files
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

##Preparing the IMDB movie reviews data

For instance, the `train/pos/` directory contains a set of `12,500` text files, each of which
contains the text body of a positive-sentiment movie review to be used as training data.
The negative-sentiment reviews live in the “neg” directories. 

In total, there are `25,000`
text files for training and another 25,000 for testing.

Next, let’s prepare a validation set by setting apart 20% of the training text files in a new directory, `aclImdb/val`:

In [4]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir/"val"
train_dir = base_dir/"train"

for category in ("neg", "pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  # Shuffle the list of training files using a seed, to ensure we get the same validation set every time we run the code
  random.Random(1337).shuffle(files)
  # Take 20% of the training files to use for validation
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    # Move the files to aclImdb/val/neg and aclImdb/val/pos
    shutil.move(train_dir/category/fname, val_dir/category/fname)

Remember how, we used the `image_dataset_from_directory` utility to
create a batched Dataset of images and their labels for a directory structure? You can do the exact same thing for text files using the `text_dataset_from_directory` utility.

Let’s create three Dataset objects for training, validation, and testing:

In [5]:
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


These datasets yield inputs that are TensorFlow `tf.string` tensors and targets that are `int32` tensors encoding the value “0” or “1.”

In [6]:
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Comparable to Fight Club, The Matrix, A.I., Sixth Sense, among others. This film approaches the psyche in a way never done before. The first 30 minutes builds a interesting love story between Diaz-Cruise-Cruz. The rest of the movie is, well, confusing, you'll pick more every time you watch it (i've gone to the movies to see it 3 times now)", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


###Preparing datasets for sequences

First, let’s prepare datasets that return integer sequences.

In [7]:
max_length = 600
max_tokens = 20000

# Prepare a dataset that only yields raw text inputs (no labels).
text_only_train_ds = train_ds.map(lambda x, y: x)

# This is a reasonable choice, since the average review length is 233 words, and only 5% of reviews are longer than 600 words.
text_vectorization = layers.TextVectorization(max_tokens=max_tokens,
                                              output_mode="int",
                                              output_sequence_length=max_length)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

##Transformer encoder

Let’s implement a Transformer encoder and try it on the movie review sentiment
classification task.

In [8]:
class TransformerEncoder(layers.Layer):
  
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim    # Size of the input token vectors
    self.dense_dim = dense_dim    # Size of the inner dense layer
    self.num_heads = num_heads  # Number of attention heads

    self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    self.dense_projection = keras.Sequential([
         layers.Dense(dense_dim, activation="relu"),
         layers.Dense(embed_dim)                                     
    ])

    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    # The mask that will be generated by the Embedding layer will be 2D, but the attention layer expects to be 3D or 4D, so we expand its rank.
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(inputs, inputs, attention_mask=mask)
    projection_input = self.layernorm_1(inputs + attention_output)
    projection_output = self.dense_projection(projection_input)
    return self.layernorm_2(projection_input + projection_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim
    })
    return config

Because `BatchNormalization` doesn’t work well for sequence data. Instead, we’re using the layer, `LayerNormalization` which normalizes each sequence independently from other sequences in the batch.

```python
def layer_normalization(batch_of_sequences):
  # Input shape: (batch_size, sequence_length, embedding_dim)
  # To compute mean and variance, we only pool data over the last axis (axis -1).
  mean = np.mean(batch_of_sequences, keepdims=True, axis=-1)
  variance = np.var(batch_of_sequences, keepdims=True, axis=-1)
  return (batch_of_sequences - mean) / variance
```

Compare to BatchNormalization (during training):

```python
def batch_normalization(batch_of_images):
  # Input shape: (batch_size, height, width, channels)
  # Pool data over the batch axis (axis 0), which creates interactions between samples in a batch.
  mean = np.mean(batch_of_images, keepdims=True, axis=(0, 1, 2))
  variance = np.var(batch_of_images, keepdims=True, axis=(0, 1, 2))
  return (batch_of_images - mean) / variance
```

While collects information from many samples to obtain accu
`BatchNormalization` rate statistics for the feature means and variances,
pools data `LayerNormalization` within each sequence separately, which is more appropriate for sequence data.

Now that we’ve implemented our TransformerEncoder, we can use it to assemble a
text-classification model.



In [9]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

In [ ]:
# One input is a sequence of integers
inputs = keras.Input(shape=(None, ), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
# Since TransformerEncoder returns full sequences, we need to reduce each sequence to a single vector for classification via a global pooling layer.
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_average_pooling1d_1   (None, 256)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257 

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("transformer_encoder.keras", save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callbacks)

In [ ]:
# Provide the custom TransformerEncoder class to the model-loading process.
model = tf.keras.models.load_model("transformer_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 636s 812ms/step - loss: 0.2987 - accuracy: 0.8764
Test accuracy: 0.876


It gets to 87.5% test accuracy—slightly worse than the GRU model.

`Self-attention` is a set-processing mechanism, focused on the relationships between pairs of sequence elements—it’s blind to whether these elements occur at the beginning, at the end, or in the middle of a sequence.

`Transformer` was a hybrid approach that is technically order-agnostic, but that manually injects order information in the representations it processes. 

This is the missing ingredient! It’s called `positional encoding`.

##Re-inject order information using positional encoding

The idea behind positional encoding is very simple: to give the model access to wordorder information.

The simplest scheme you could come up with would be to concatenate the word’s
position to its embedding vector. You’d add a “position” axis to the vector and fill it with 0 for the first word in the sequence, 1 for the second, and so on.

That may not be ideal, however, because the positions can potentially be very large
integers, which will disrupt the range of values in the embedding vector. As you know,
neural networks don’t like very large input values, or discrete input distributions.

The original “Attention is all you need” paper used an interesting trick to encode
word positions: it added to the word embeddings a vector containing values in the
range `[-1, 1]` that varied cyclically depending on the position (it used cosine functions
to achieve this). This trick offers a way to uniquely characterize any integer in a
large range via a vector of small values.

We’ll do something simpler and more effective: we’ll learn positionembedding
vectors the same way we learn to embed word indices. We’ll then proceed
to add our position embeddings to the corresponding word embeddings, to obtain a
position-aware word embedding. This technique is called `positional embedding`.



In [16]:
class PositionalEmbedding(layers.Layer):

  # A downside of position embeddings is that the sequence length needs to be known in advance
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)

    # Prepare an Embedding layer for the token indices.
    self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
    # And another one for the token positions
    self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)

    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    # add both embedding vectors together
    return embedded_tokens + embedded_positions

  def compute_mask(self, inputs, mask=None):
    """
    Like the Embedding layer, this layer should be able to generate a mask so we can ignore padding 0s in the inputs. 
    The compute_mask method will called automatically by the framework, and the mask will get propagated to the next layer.
    """
    return tf.math.not_equal(inputs, 0)

  def get_config(self):
    config = super().get_config()
    config.update({
        "output_dim": self.output_dim,
        "sequence_length": self.sequence_length,
        "input_dim": self.input_dim
    })
    return config

You would use this `PositionEmbedding` layer just like a regular `Embedding` layer.

##A text classification Transformer

All you have to do to start taking word order into account is swap the old `Embedding` layer with our position-aware version.

In [11]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

In [17]:
# One input is a sequence of integers
inputs = keras.Input(shape=(None, ), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
# Since TransformerEncoder returns full sequences, we need to reduce each sequence to a single vector for classification via a global pooling layer.
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_1 (Pos  (None, None, 256)        5273600   
 itionalEmbedding)                                               
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_average_pooling1d_1   (None, 256)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

In [18]:
callbacks = [keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras", save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 46s 70ms/step - loss: 0.4445 - accuracy: 0.7965 - val_loss: 0.2685 - val_accuracy: 0.8902
Epoch 2/20
625/625 [==============================] - 44s 70ms/step - loss: 0.2463 - accuracy: 0.9057 - val_loss: 0.2561 - val_accuracy: 0.8968
Epoch 3/20
625/625 [==============================] - 43s 69ms/step - loss: 0.1892 - accuracy: 0.9286 - val_loss: 0.2532 - val_accuracy: 0.9010
Epoch 4/20
625/625 [==============================] - 44s 69ms/step - loss: 0.1577 - accuracy: 0.9407 - val_loss: 0.2781 - val_accuracy: 0.8908
Epoch 5/20
625/625 [==============================] - 44s 69ms/step - loss: 0.1329 - accuracy: 0.9518 - val_loss: 0.2761 - val_accuracy: 0.8912
Epoch 6/20
625/625 [==============================] - 44s 69ms/step - loss: 0.1138 - accuracy: 0.9596 - val_loss: 0.3861 - val_accuracy: 0.8876
Epoch 7/20
625/625 [==============================] - 45s 71ms/step - loss: 0.0963 - accuracy: 0.9678 - val_loss: 0.3837 - val_accuracy:

In [19]:
# Provide the custom TransformerEncoder class to the model-loading process.
model = tf.keras.models.load_model("full_transformer_encoder.keras", 
                                   custom_objects={"TransformerEncoder": TransformerEncoder, "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 20s 25ms/step - loss: 0.2898 - accuracy: 0.8846
Test accuracy: 0.885


We get to 88.3% test accuracy, a solid improvement that clearly demonstrates the value of word order information for text classification. This is our best sequence model so far!